In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("Spark NLP")\
    .master("local[*]")\
    .config("spark.driver.memory","12g") \
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
    .config("spark.driver.maxResultSize", "0") \
    .config("spark.kryoserializer.buffer.max", "2000M")\
    .config("spark.jars.packages", "com.microsoft.sqlserver:mssql-jdbc:9.4.0.jre8") \
    .getOrCreate()

:: loading settings :: url = jar:file:/usr/local/spark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/worker2/.ivy2/cache
The jars for the packages stored in: /home/worker2/.ivy2/jars
com.microsoft.sqlserver#mssql-jdbc added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-26921a98-26aa-4fdb-ae46-639e22d22063;1.0
	confs: [default]
	found com.microsoft.sqlserver#mssql-jdbc;9.4.0.jre8 in central
:: resolution report :: resolve 226ms :: artifacts dl 5ms
	:: modules in use:
	com.microsoft.sqlserver#mssql-jdbc;9.4.0.jre8 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   1   |   0   |   0   |   0   ||   1   |   0   |
	---------------------------------------------------------------------
:: retrieving :: org.apache.spark#spark-submit-parent-26921a98-

[ WARN] Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
[ WARN] Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [61]:
CONNECTOR_TYPE='com.microsoft.sqlserver.jdbc.SQLServerDriver'
SQL_USERNAME='tyeson'
SQL_PASSWORD='Canhtoan1509'
SQL_SERVERNAME='tyeson.database.windows.net'
SQL_DBNAME='DW'
CONNECTION_STRING_SOURCE='jdbc:sqlserver://tyeson.database.windows.net:1433;databaseName=DW;'
CONNECTION_STRING_TARGET='jdbc:sqlserver://tyeson.database.windows.net:1433;databaseName=DM_CAREER_LO;'

In [25]:
dim_career = spark.read.format('jdbc')\
    .option("url", CONNECTION_STRING_SOURCE)\
    .option('driver',CONNECTOR_TYPE) \
    .option('dbtable','dim_career')\
    .option("user", SQL_USERNAME)\
    .option('password', SQL_PASSWORD)\
    .load()

In [26]:
from pyspark.sql.functions import col
dim_career =dim_career.select(col('careerId').alias('careerId_dim'), 'careerName')

In [5]:
dim_competency = spark.read.format('jdbc')\
    .option("url", CONNECTION_STRING_SOURCE)\
    .option('driver',CONNECTOR_TYPE) \
    .option('dbtable','dim_competency')\
    .option("user", SQL_USERNAME)\
    .option('password', SQL_PASSWORD)\
    .load()

In [27]:
fact_jobposting = spark.read.format('jdbc')\
    .option("url", CONNECTION_STRING_SOURCE)\
    .option('driver',CONNECTOR_TYPE) \
    .option('dbtable','fact_jobposting')\
    .option("user", SQL_USERNAME)\
    .option('password', SQL_PASSWORD)\
    .load()

In [28]:
fact_jobposting = fact_jobposting.select('jobKey', 'jobpostDate', col('careerId').alias('careerId'), 'competencyId')

In [ ]:
df_career_competency = fact_jobposting.join(dim_career, col('careerId') == col('careerId_dim'), 'inner').select('jobKey', 'jobpostDate', 'careerId', 'careerName', 'competencyId')

In [33]:
competency = dim_competency.select(col('competencyId').alias('comId'), 'competencyName', 'type')

In [36]:
df_full = df_career_competency.join(competency, col('competencyId') == col('comId'), 'inner').select('*')
df_full = df_full.select('jobKey', 'jobpostDate', 'careerId', 'careerName', 'competencyId', 'competencyName', 'type')

In [63]:
dim_competency.write.format('jdbc')\
    .mode("Append")\
    .option("url", CONNECTION_STRING_TARGET)\
    .option('driver',CONNECTOR_TYPE) \
    .option('dbtable','dim_competency')\
    .option("user", SQL_USERNAME)\
    .option('password', SQL_PASSWORD)\
    .save()

In [50]:
career_dis = df_career_competency.dropDuplicates(['jobKey', 'jobpostDate', 'careerId', 'careerName']).select('jobKey', 'jobpostDate', 'careerId', 'careerName')

In [93]:
career = career_dis.select('careerId', 'careerName').groupBy('careerId', 'careerName').count().orderBy('careerId')

In [ ]:
import pyspark.sql.functions as f
from pyspark.sql.functions import trim,ltrim,rtrim
from pyspark.sql.functions import split, col, monotonically_increasing_id
career = career.select(monotonically_increasing_id().alias('careerKey'), 'careerId', 'careerName', col('count').alias('numpost'), f.current_timestamp().alias('createDate'), f.current_timestamp().alias('expiredDate'), f.lit(1).alias('rowStatus'))

In [62]:
career.write.format('jdbc')\
    .mode("Append")\
    .option("url", CONNECTION_STRING_TARGET)\
    .option('driver',CONNECTOR_TYPE) \
    .option('dbtable','dim_career')\
    .option("user", SQL_USERNAME)\
    .option('password', SQL_PASSWORD)\
    .save()

In [101]:
fact = df_full.select(col('careerId').alias('careerId_fact'), 'competencyId').groupBy('careerId_fact', 'competencyId').count()

In [106]:
fact_last = fact.join(career, fact['careerId_fact'] == career['careerId'], 'inner').select('*').orderBy('careerKey')

In [107]:
fact_last = fact_last.select('careerKey', 'competencyId', col('count').alias('numRequired'))

In [108]:
fact_last.write.format('jdbc')\
    .mode("Append")\
    .option("url", CONNECTION_STRING_TARGET)\
    .option('driver',CONNECTOR_TYPE) \
    .option('dbtable','career_need_competenc')\
    .option("user", SQL_USERNAME)\
    .option('password', SQL_PASSWORD)\
    .save()

In [109]:
spark.stop()